# Composable Objects
- Pros:
  - Can fully customize:
    - query engine
    - retriever
    - query pipeline
    - another node
  - Can save and load data easily

In [10]:
from llama_index.core import Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

ollama_embedding = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
)

ollama = Ollama(
    model="llama3.2:3b-instruct-fp16",
    base_url="http://localhost:11434"
)

Settings.llm = ollama
Settings.embed_model = ollama_embedding

In [3]:
from llama_index.core import download_loader

from llama_index.readers.file import PyMuPDFReader

llama2_docs = PyMuPDFReader().load_data(
    file_path="../documents/ods-cpp (1).pdf", metadata=True
)
attention_docs = PyMuPDFReader().load_data(
    file_path="../documents/RAT.pdf", metadata=True
)

In [4]:
from llama_index.core.node_parser import TokenTextSplitter

nodes = TokenTextSplitter(
    chunk_size=1024, chunk_overlap=128
).get_nodes_from_documents(llama2_docs + attention_docs)

In [6]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.storage.docstore.firestore import FirestoreDocumentStore
from llama_index.storage.docstore.dynamodb import DynamoDBDocumentStore

docstore = SimpleDocumentStore()
docstore.add_documents(nodes)

In [12]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

client = QdrantClient(path="../composable/qdrant_data")
vector_store = QdrantVectorStore("composable", client=client)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(nodes=nodes)
vector_retriever = index.as_retriever(similarity_top_k=2)
bm25_retriever = BM25Retriever.from_defaults(
    docstore=docstore, similarity_top_k=2
)

In [13]:
from llama_index.core.schema import IndexNode

vector_obj = IndexNode(
    index_id="vector", obj=vector_retriever, text="Vector Retriever"
)
bm25_obj = IndexNode(
    index_id="bm25", obj=bm25_retriever, text="BM25 Retriever"
)

In [14]:
from llama_index.core import SummaryIndex

summary_index = SummaryIndex(objects=[vector_obj, bm25_obj])

In [15]:
query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize", verbose=True
)

In [16]:
response = await query_engine.aquery(
    "What is the difference between stack and queue?"
)

Retrieval entering vector: VectorIndexRetriever
Retrieval entering bm25: BM25Retriever


In [17]:
print(str(response))

A stack and a queue are both data structures that follow a specific ordering of elements, but they differ in how elements are added and removed.

In a stack, elements are added and removed from one end, known as the top or apex. This means that the most recently added element is always at the top of the stack and is the first one to be removed when the pop operation is performed.

On the other hand, a queue is a collection of elements where elements are added to one end and removed from the other end. The order in which elements are added determines the order in which they will be removed. This means that an element added early in the queue will be removed before an element added later.

To illustrate this difference, consider a stack as a vertical pile of plates. When you add a new plate to the top of the pile, it becomes the topmost plate and is the first one to be removed when you take a plate off the top. In contrast, a queue is like a line of people waiting for service at a counte

In [18]:
# qdrant is already saved automatically!
# we only need to save the docstore here

# save our docstore nodes for bm25
docstore.persist("./docstore.json")

## Load

In [19]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient

docstore = SimpleDocumentStore.from_persist_path("./docstore.json")

client = QdrantClient(path="./qdrant_data")
vector_store = QdrantVectorStore("composable", client=client)

In [20]:
from llama_index.core.schema import IndexNode

vector_obj = IndexNode(
    index_id="vector", obj=vector_retriever, text="Vector Retriever"
)
bm25_obj = IndexNode(
    index_id="bm25", obj=bm25_retriever, text="BM25 Retriever"
)